In [2]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser

In [78]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
filing_links = pd.read_excel(
    "/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC__sec_filing_links.xlsx")

path = '/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, sheet_name=None)

In [79]:
dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
for sheet_name, sheet_df in dataframes.items():
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

DataFrame name: March_31_2013 : (352, 7)
DataFrame name: June_30_2013 : (376, 7)
DataFrame name: September_30_2013 : (368, 7)
DataFrame name: December_31_2013 : (17, 7)
DataFrame name: March_31_2014 : (415, 7)
DataFrame name: June_30_2014 : (406, 7)
DataFrame name: September_30_2014 : (416, 7)
DataFrame name: December_31_2014 : (19, 7)
DataFrame name: March_31_2015 : (415, 7)
DataFrame name: June_30_2015 : (422, 7)
DataFrame name: September_30_2015 : (437, 7)
DataFrame name: December_31_2015 : (18, 7)
DataFrame name: March_31_2016 : (472, 7)
DataFrame name: June_30_2016 : (476, 7)
DataFrame name: September_30_2016 : (480, 7)
DataFrame name: December_31_2016 : (20, 7)
DataFrame name: March_31_2017 : (669, 7)
DataFrame name: June_30_2017 : (678, 7)
DataFrame name: September_30_2017 : (746, 7)
DataFrame name: December_31_2017 : (766, 7)
DataFrame name: March_31_2018 : (794, 7)
DataFrame name: June_30_2018 : (769, 7)
DataFrame name: September_30_2018 : (771, 7)
DataFrame name: December_31_

In [80]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d %Y")
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],November 04 2010,September 30 2010,https://www.sec.gov/Archives/edgar/data/128775...
1,10-K,"Annual report [Section 13 and 15(d), not S-K I...",March 01 2011,December 31 2010,https://www.sec.gov/Archives/edgar/data/128775...
2,10-Q,Quarterly report [Sections 13 or 15(d)],May 03 2011,March 31 2011,https://www.sec.gov/Archives/edgar/data/128775...
3,10-Q,Quarterly report [Sections 13 or 15(d)],August 04 2011,June 30 2011,https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],November 08 2011,September 30 2011,https://www.sec.gov/Archives/edgar/data/128775...


In [81]:

matches = []  # To store rows of matching data

for file, frame in dataframes.items():
    if frame.shape[0] < 100:
        print(file.replace('_', ' '))
        matching_rows = filing_links[filing_links['Reporting date'].str.replace(
            ' ', '') == file.replace('_', '')]
        if not matching_rows.empty:
            matches.append(matching_rows)
            print(matching_rows['Filings URL'].values[0])
            # webbrowser.open(matching_rows['Filings URL'].values[0])
working_class = pd.concat(matches, ignore_index=True)

December 31 2013
https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm
December 31 2014
https://www.sec.gov/Archives/edgar/data/1287750/000104746915001240/a2222984z10-k.htm
December 31 2015
https://www.sec.gov/Archives/edgar/data/1287750/000104746916010353/a2227293z10-k.htm
December 31 2016
https://www.sec.gov/Archives/edgar/data/1287750/000104746917000808/a2230898z10-k.htm


In [3]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')

    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass

    for linebreak in soup.find_all('br'):
        linebreak.extract()
    for linebreak in soup.find_all('hr'):
        linebreak.extract()

    return soup

In [83]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746911001575/a2202281z10-k.htm'
# # date = 'December 31, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465910055721/a10-17362_110q.htm'
# date = 'September 30, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
date = 'September 30, 2013'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        # print(len(soup_content.find_all(text=consolidated_schedule_regex)))
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)
                    print("ran")

                    # if html_table.find(text=re.compile(r'Other than', re.IGNORECASE)):
                    #     print(html_table.find(text=re.compile(
                    #         r'Other than', re.IGNORECASE)))
                    #     break
                    # elif html_table.find_next(text='Derivative Instruments'):
                    #     print(html_table.find_next(
                    #         text='Derivative Instruments'))
                    #     break
                    # elif html_table.find_next(text=consolidated_schedule_regex):
                    #     next_line_text = tag.find_next(
                    #         text=re.compile(date_regex_pattern)).text
                    #     date_str2 = re.search(
                    #         date_regex_pattern, next_line_text)
                    #     if date_str2.group(1) == date_str:
                    #         break
                    print(html_table.find_previous_sibling())
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table.to_csv('example.csv')
master_table.to_excel('example.xlsx')

master_table

ran
<p><b><font> </font></b></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><b><font> </font></b></p>
ran
<p><font> </font></p>
ran
<p><b><font> </font></b></p>
ran
<p><b><font> </font></b></p>
ran
<p><b><font> </font></b></p>
ran
<p><b><font> </font></b></p>
ran
<p><b><font> </font></b></p>
ran
None
ran
None
ran
None
ran
None
ran
None
ran
None
ran
None
ran
None
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font></p>
ran
<p><font> </font>

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Company(1),NaN,Business Description,NaN,Investment,NaN,Interest(5)(11),NaN,Acquisition Date,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Investment Funds and Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"AGILE Fund I, LLC (9)",NaN,Investment partnership,NaN,Member interest (0.50% interest),NaN,NaN,NaN,4/1/2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"CIC Flex, LP (9)",NaN,Investment partnership,NaN,Limited partnership units (0.94 units),NaN,NaN,NaN,9/7/2007,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Covestia Capital Partners, LP (9)",NaN,Investment partnership,NaN,Limited partnership interest (47.00% interest),NaN,NaN,NaN,6/17/2008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1882,NaN,NaN,Michael J. Arougheti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1883,NaN,NaN,Chief Executive Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1884,"Dated: November 5, 2013",By,/s/ Penni F. Roll,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1885,NaN,NaN,Penni F. Roll,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'

# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(True):
        text = tag.get_text(strip=True)
        if re.fullmatch(date_regex_pattern2, text):
            date_str = re.search(date_regex_pattern1, text)
            print(tag)
            print(date_str)
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                html_table = tag.find_next('table')
                print('Table found: ')
                html_table = tag.find_next('table')
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)

    return master_table


master_table = extract_tables(content, qtr_date)
# master_table.to_csv('example.csv')
# master_table.to_excel('example.xlsx')

master_table

<tr>
<th><font> </font></th>
<th><font> </font></th>
<th><font><b>As of December 31, 2013 </b></font></th>
<th><font> </font></th>
</tr>
<re.Match object; span=(6, 23), match='December\xa031, 2013'>
Table found: 
<th><font><b>As of December 31, 2013 </b></font></th>
<re.Match object; span=(6, 23), match='December\xa031, 2013'>
Table found: 
<font><b>As of December 31, 2013 </b></font>
<re.Match object; span=(6, 23), match='December\xa031, 2013'>
Table found: 
<b>As of December 31, 2013 </b>
<re.Match object; span=(6, 23), match='December\xa031, 2013'>
Table found: 
<tr>
<th><font> </font></th>
<th><font> </font></th>
<th><font><b>As of December 31, 2012 </b></font></th>
<th><font> </font></th>
</tr>
<re.Match object; span=(6, 23), match='December\xa031, 2012'>
<th><font><b>As of December 31, 2012 </b></font></th>
<re.Match object; span=(6, 23), match='December\xa031, 2012'>
<font><b>As of December 31, 2012 </b></font>
<re.Match object; span=(6, 23), match='December\xa031, 2012'>
<b>As 

/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/140491150.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/140491150.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/140491150.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,NaN,"As of December 31, 2012",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,Unobservable Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Asset Category,NaN,Fair Value,NaN,Primary Valuation Technique,NaN,Input,NaN,Range,NaN,Weighted Average,NaN,NaN,NaN
3,First lien senior secured loans,NaN,NaN,2321272,NaN,Yield Analysis,NaN,Market Yield,NaN,5.3% - 21.5%,NaN,NaN,9.2,%
4,Second lien senior secured loans,NaN,NaN,1233872,NaN,Yield Analysis,NaN,Market Yield,NaN,8.3% - 21.9%,NaN,NaN,10.9,%
5,Subordinated certificates of the SSLP,NaN,NaN,1263644,NaN,Discounted Cash Flow,NaN,Discount Rate,NaN,11.5% - 14.5%,NaN,NaN,13.5,%
6,Senior subordinated debt,NaN,NaN,259820,NaN,Yield Analysis,NaN,Market Yield,NaN,10.0% - 18.6%,NaN,NaN,14.9,%
7,Preferred equity securities,NaN,NaN,250118,NaN,EV Market Multiple Analysis,NaN,EBITDA Multiple,NaN,4.5x - 10.5x,NaN,NaN,8.1x,NaN
8,Other equity securities and other,NaN,NaN,585931,NaN,EV Market Multiple Analysis,NaN,EBITDA Multiple,NaN,4.5x - 12.8x,NaN,NaN,7.4x,NaN
9,Total,NaN,NaN,5914657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
# this one works for all other
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None

    for tag in soup_content.find_all(True):
        text = tag.get_text(strip=True)
        if re.fullmatch(date_regex_pattern2, text):
            date_str = re.search(date_regex_pattern1, text)
            next_line = tag.find_next().text
            if re.search('dollar', next_line):
                print(tag)
                print(True)
                print(date_str.group(1))
                if date_str is not None:
                    date_str = str(date_str.group(1))
                    date_str = unicodedata.normalize('NFKD', date_str)
                    # print(date_str)
                if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                    # print('match')
                    html_table = tag.find_next('table')
                    print('Table found: ')
                    html_table = tag.find_next('table')
                    if master_table is None:
                        master_table = pd.read_html(
                            html_table.prettify(), skiprows=0, flavor='bs4')[0]
                        master_table = master_table.applymap(lambda x: unicodedata.normalize(
                            'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                        master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                                  regex=True)
                        master_table = master_table.dropna(how='all', axis=0)
                    else:
                        new_table = pd.read_html(
                            html_table.prettify(), skiprows=0, flavor='bs4')[0]
                        new_table = new_table.applymap(lambda x: unicodedata.normalize(
                            'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                        new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                            regex=True)
                        new_table = new_table.dropna(how='all', axis=0)
                        # print('head')
                        # print(new_table.head()) # text
                        master_table = master_table.append(
                            new_table.dropna(how='all', axis=0).reset_index(
                                drop=True).drop(index=0),
                            ignore_index=True)
    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table.to_csv('example.csv')
master_table.to_excel('example.xlsx')
master_table

AttributeError: 'NoneType' object has no attribute 'applymap'

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None

    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):

        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text

        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            print(date_str.group(1))
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
                # print(date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                # print(tag)
                # print(find_next)
                # print(next_line)
                # print('match')
                html_table = tag.find_next('table')
                print('Table found: ')
                html_table = tag.find_next('table')
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)
    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table.to_csv('example.csv')
master_table.to_excel('example.xlsx')
# master_table

December 31, 2017
Table found: 
December 31, 2017
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/154152187.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/154152187.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


December 31, 2017
Table found: 
December 31, 2017
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/154152187.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/154152187.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


December 31, 2017
Table found: 
December 31, 2017
Table found: 


...

December 31, 2017
Table found: 
December 31, 2017
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/154152187.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/154152187.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


December 31, 2017
Table found: 
December 31, 2017
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_1574/154152187.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
December 31, 2016
(997, 20)


In [ ]:
def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    # print('1: ' + str(soi_table_df.shape))

    # Separate header and data
    soi_table_header = soi_table_df.iloc[0].dropna(how='any')
    soi_table_data_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    # print('2: ' + str(soi_table_data_df.shape))

    # drops all the rows that contains header
    soi_table_data_df = soi_table_data_df[soi_table_data_df[soi_table_data_df.columns[0]]
                                          != soi_table_data_df.columns[0]]
    # print('3: ' + str(soi_table_data_df.shape))

    # Drop rows with only two non-null values becuase all the subtotal contain 2 value only
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)
    # print('4: ' + str(soi_table_data_df.shape))

    # replace all the - in the data table with 0
    soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)

    # fix the all the nan value column , Amortized Cost, Fair Value
    columns_to_fill = ['Amortized Cost', 'Fair Value']
    for col in columns_to_fill:
        col_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc(col.replace(' ', ''))
        next_col_index = col_index + 1
        for i in range(len(soi_table_data_df)):
            current_value = soi_table_data_df.iat[i, col_index]
            if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
                next_valid_index = next((j for j, v in enumerate(
                    soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

                if next_valid_index is not None:
                    next_value = soi_table_data_df.iat[i, next_valid_index]
                    soi_table_data_df.iat[i, col_index] = next_value
                    soi_table_data_df.iat[i, next_valid_index] = pd.NA

    # drops everything after FairValue
    if 'FairValue' in soi_table_data_df.columns.str.replace(' ', ''):
        start_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc('FairValue')
        soi_table_data_df = soi_table_data_df.iloc[:, :start_index+1]

    # Drop rows with only two non-null values this one recheacks
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)

    # Forward fill the first two columns
    col_indices = [0, 1]
    soi_table_data_df.iloc[:, col_indices] = soi_table_data_df.iloc[:, col_indices].fillna(
        method='ffill')
    # print('7: ' + str(soi_table_data_df.shape))

    # Drop rows with all missing values
    soi_table_df = soi_table_df.dropna(how='all')
    # print('5: ' + str(soi_table_data_df.shape))

    # Drop columns with all missing values
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    # print('6: ' + str(soi_table_data_df.shape))

    # fill all the nan with 0
    soi_table_data_df = soi_table_data_df.fillna(0)

    # converts to number
    # cols_to_convert = ['Shares/Units', 'Principal',
    #                    'Amortized Cost', 'Fair Value']
    # for col in cols_to_convert:
    #     if col.replace(' ', '') in soi_table_data_df.columns.str.replace(' ', ''):
    #         col_index = soi_table_data_df.columns.str.replace(
    #             ' ', '').get_loc(col.replace(' ', ''))
    #         soi_table_data_df.iloc[:, col_index] = pd.to_numeric(
    #             soi_table_data_df.iloc[:, col_index], errors='coerce').fillna(0)

    cols_to_convert = ['Shares/Units', 'Principal',
                       'Amortized Cost', 'Fair Value']
    for col in cols_to_convert:
        if col.replace(' ', '') in soi_table_data_df.columns.str.replace(' ', ''):
            col_index = soi_table_data_df.columns.str.replace(
                ' ', '').get_loc(col.replace(' ', ''))
            converted_data = pd.to_numeric(
                soi_table_data_df.iloc[:, col_index], errors='coerce').fillna(0)
            soi_table_data_df[soi_table_data_df.columns[col_index]
                              ] = converted_data

    soi_table_data_df = soi_table_data_df.reset_index(drop=True)

    # keeping the total for now to check if the total is right
    # Drop rows based on regex pattern (e.g., 'subtotal' or 'total')
    # pattern = r'^([Ss]ubtotal)|([Tt]otal)'
    # mask = soi_table_data_df.apply(lambda row: row.astype(
    #     str).str.contains(pattern, case=False, na=False)).any(axis=1)
    # soi_table_data_df = soi_table_data_df[~mask]
    # # print('4: ' + str(soi_table_data_df.shape))

    # we dont need this here because we are dropping subtotal before
    # Drop rows labeled as subtotals
    # subtotal_rows = soi_table_data_df[soi_table_data_df['Company (1)'].str.replace(' ', '').str.contains(
    #     'subtotal', case=False, na=False)]
    # soi_table_data_df = soi_table_data_df[~soi_table_data_df.index.isin(
    #     subtotal_rows.index)]
    # print('3: ' + str(soi_table_data_df.shape))

    # we dont need this here because we are dropping subtotal before
    # columns_to_drop = []
    # for column in soi_table_data_df.columns:
    #     # Check for NaN values in the column
    #     # Use .item() to get a single boolean value
    #     if soi_table_data_df[column].isna().any().item():
    #         columns_to_drop.append(column)
    # soi_table_data_df.drop(columns=columns_to_drop, inplace=True)

    # print('Final: ' + str(soi_table_data_df.shape))

    return soi_table_data_df


process_table_ = process_table(master_table, "")

In [36]:

headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None

    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):

        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text
        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            print(date_str.group(1))
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
                # print(date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                # print(tag)
                # print(find_next)
                # print(next_line)
                # print('match')
                html_table = tag.find_next('table')
                print('Table found: ')
                # while soup_content.find_next(text=re.compile(date_regex_pattern2)) is not False:
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)

                # while soup_content.find_next(text=re.compile(date_regex_pattern2)) is not False:
                # while html_table.find_next('table'):
                #     print(html_table)
                #     if soup_content.find(text=re.compile(date_regex_pattern2)):
                #         print('broke')
                #         break
                    html_table = html_table.find_next('table')

    # master_table = master_table.applymap(
    #     lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    # master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
    #     r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    # print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
# master_table.to_csv('example.csv')
# master_table.to_excel('example.xlsx')
# master_table

December 31, 2013
Table found: 
December 31, 2012


In [42]:
# all while

headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None
    dates = set()
    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):

        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text
        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
                dates.add(date_str)
    


master_table = extract_tables(content, qtr_date)

{'December 31, 2013', 'December 31, 2012'}
